The goal of the project is to create a functioning code that will extract information from a pH sensor, store it in a storage medium using pandas, and then plot the data into a titration curve for analysis.

Levi: My part of the project is to create a code that works on the interpretation of data from various parts of the research to create a final resulating pH. The URL to the repository is attached: 
https://github.com/nmalmberg/Genesys/blob/master/genesys.py
https://github.com/AtlasScientific/Raspberry-Pi-sample-code/blob/master/uart.py

Caden: My part of the project is to take the data that was collected from the pH sensor and store it in a storage medium using pandas. This storage medium will then be used to create a titration curve for the experiment. I have used a pandas user guide to aid in the progress of my project. The URL for this information is https://pandas.pydata.org/docs/user_guide/io.html.
Atlatas Scientific pH-EZO datasheet manual
https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html

Rylie: My part of the project is to take the collected data and plot it into a titration curve using the Matplot code. The progress I have made is the basic code for the axes with appropriate labels. The data values included are not the data points used in this experiment, but placeholders until data values are taken. The information and code was obtained from https://www.w3schools.com/python/matplotlib_labels.asp.

This code block creates a connection to the pH-EZO sensor

In [41]:
import time
import serial

#connecting to usbport
usbport = '/dev/cu.usbserial-14210'
ser = serial.Serial(usbport, 9600, timeout=0)

pH_min = 0.001
pH_max = 14.000

def pH_range(pH, pH_min, pH_max):
    if (pH < pH_min) or (pH > pH_max):
        return False
        
    else:
        return True
    
ser.write(b'C,0\n')
ser.write(b'R\n')
ser.read(7)

b'594\r3.5'

This code defines the calibration of the pH sensor

In [37]:
#Calibration values set
CALIBRATION_VALUES = [10, 7, 4]
# creating a function to create a calibration mode over the sensor #
def calibrate():
    ser.write(b'C\n')
    time.sleep(0.1)
    ser.write(b'R\n')
    time.sleep(0.1)
    ser.write(b'W'+bytes(CALIBRATION_VALUES)+b'\n')
    time.sleep(0.1)
    ser.write(b'S\n')
    time.sleep(0.1)
    ser.write(b'R\n')
    time.sleep(0.1)

calibrate()

This code defines the readings of the pH sensor into a loop for collection

In [42]:
def read_ph():
    ser.write(b'C,0\n')
    
    ser.write(b'R\n')
    time.sleep(0.1)

    response = ser.read(7)

    response = str(response, 'utf-8').strip()
    response = ''.join(filter(str.isdigit, response))

    correct = re.search(r'(\d+(?:\.\d+)?)', response)
        if correct:
            pH = float(correct.group(1))
            return pH
        else:
            return None  
    
    pH = float(response)
    return pH
    
# main loop to read pH value of the sensor # 
while True:
    pH = read_ph()
    print("pH:", pH)
    time.sleep(3)

# termination of loop #
stop_loop = False
loop_time_limit = 60
start_time = time.time()
while not stop_loop:
    elapsed_time = time.time() - start_time
    if elapsed_time >= loop_time_limit:
        stop_loop = True
    else:
        time.sleep(1)

pH: 94359.0
pH: 43594.0
pH: 3593.0
pH: 35933.0


KeyboardInterrupt: 

This code block takes data from the sensor and converts into appropriate data frame to use for analysis

In [7]:
##Read the data collected then it will convert into a dataframe then will be read to be plotted 

import pandas as pd
from io import StringIO

data = pd.read_csv('ph_ezo_data.txt', delimiter='\t', names=['Time', 'pH'])

# Convert the 'Time' column to a DateTime object
df['Time'] = pd.to_datetime(df['Time'], format='%Y-%m-%d %H:%M:%S.%f')

#Naming the acid that was used in titration
Acid = input("Enter the name of the Acid being titrated: ")

#Convert to the volume of NaOH from the time that passed
Volume = float(input("Enter the volume of NaOH added to the acid: ")) #The volume used for the titration

flow_rate = Volume / loop_time_limit #Flow Rate of the titration
NaOH = flow_rate * elapsed_time #NaOh added at specific elapsed time

##Dataframe that is stacked will columns of only the volume of NaOH and pH value of the solution
df = pd.DataFrame({
    'Acid': Acid, 
    'NaOH': [0], 
    'pH': df['pH']
})

df.set_index(['Time'], inplace=True)

#Adds the NaOH values to the data frame
df['NaOH'] = (flow_rate * df.index.to_series().dt.total_seconds()).values

##titration is a data frame that shows the values of the volume of NaOH and values of pH from the titration experiment
def titration(df):
    titration=pd.DataFrame(df.to_records())
    titration.columns['NaOH', 'pH']
    return titration

titration_data = titration(df)


/var/folders/2m/b8wn0zfs775d9yg80drd5w4w0000gn/T/ipykernel_4477/4029692955.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


FileNotFoundError: [Errno 2] No such file or directory: 'ph_ezo_data.txt'

The code block will calulate the equivalence point of the experiment and display a graph of the titration curve.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Calculate the pH change
pH_change = data['pH'].diff()

#Find the greatest change of pH
equivalence_point = pH_change.idxmax()

#The equivalence value is printed
print("The equivalence point is at index", equivalence point)

#Creating a scatter plot of the titration experiment
plt.scatter(titration_date['NaOH'], titration_data['pH'])

#Label varies axis and the title of the graph
plt.xlabel("NaOH volume")
plt.ylabel("pH value")
plt.title("Titration Curve")

plt.show()